In [11]:
import os
import xarray
import re
import pandas as pd
import json
import tempfile
import boto3
from datetime import datetime
import s3fs
import rasterio


s3_bucket = "ghgc-data-store-dev"
source_folder = 'lpjwsl-wetlandch4-daygrid-v2'
target_folder = 'lpjwsl-wetlandch4-daygrid-v2-new-units'
session = boto3.Session(profile_name="veda-smce-mfa")
raster_io_session = rasterio.env.Env(profile_name='veda-smce-mfa')

s3_client = session.client("s3")
files_processed = pd.DataFrame(columns=["source_file", "COGs_created"])
fs = s3fs.S3FileSystem(profile="veda-smce-mfa", anon=False)

In [12]:
def get_all_s3_keys(bucket, source_folder):
    """Get a list of all keys in an S3 bucket."""
    keys = []

    kwargs = {"Bucket": bucket, "Prefix": f"{source_folder}/"}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp["Contents"]:
            if obj["Key"].endswith(".tif") and "historical" not in obj["Key"]:
                keys.append(obj["Key"])

        try:
            kwargs["ContinuationToken"] = resp["NextContinuationToken"]
        except KeyError:
            break

    return keys


keys = get_all_s3_keys(s3_bucket, source_folder)

In [13]:
keys[0]

'lpjwsl-wetlandch4-daygrid-v2/ERA5/LPJ_EOSIM_L2_CH4e_dch4e_ERA5_19900101day.tif'

In [14]:
for key in keys[:10]:
    with raster_io_session:
        s3_file = s3_client.generate_presigned_url(
            "get_object", Params={"Bucket": s3_bucket, "Key": key}
        )
        with rasterio.open(s3_file) as src:
            data = src.read(1)
            data = data*86400

            with tempfile.NamedTemporaryFile() as temp_file:
                data.rio.to_raster(
                    temp_file.name,
                    driver="COG",
                )
                s3_client.upload_file(
                    Filename=temp_file.name,
                    Bucket=s3_bucket,
                    Key=f'{target_folder}/{key}',
                )
            files_processed = files_processed._append(
                {"source_file": key, "COGs_created": f'{target_folder}/{key}'},
                ignore_index=True,
            )

RasterioIOError: HTTP response code: 403